In [1]:
source ../config/config_duke.sh
source ../config/config_load_module_bedtools.sh
show_env

You are on Duke Server: HARDAC
BASE DIRECTORY:     /gpfs/fs1/data/reddylab/Kuei
PATH OF SOURCE:     /gpfs/fs1/data/reddylab/Kuei/source
PATH OF EXECUTABLE: /gpfs/fs1/data/reddylab/Kuei/exe
PATH OF ANNOTATION: /gpfs/fs1/data/reddylab/Kuei/annotation
PATH OF PROJECT:    /gpfs/fs1/data/reddylab/Kuei/GitRepo/Proj_CombEffect_ENCODE_FCC/notebooks
PATH OF RESULTS:    /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect_encode_fcc



In [2]:
which bedtools
bedtools -version

/nfs/software/helmod/apps/Core/bedtools2/2.25.0-fasrc01/bin/bedtools
bedtools v2.25.0


In [3]:
ls -d ${FD_LOG}
echo ${NODE}

/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/log
all


**TEST**

In [4]:
FD_REG=${FD_RES}/region/KS91_K562_astarrseq_peak_macs
FN_REG=KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed
ls -lh ${FD_REG}/${FN_REG}
cat ${FD_REG}/${FN_REG} | wc -l

-rw-rw-r-- 1 kk319 reddylab 5.7M Oct 17 05:28 /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/region/KS91_K562_astarrseq_peak_macs/KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed
247520


In [9]:
PREFIX=Tewhey_K562_TileMPRA
FD_CNT=${FD_RES}/${PREFIX}/fragment_count

ls ${FD_CNT}/*round.bed | xargs -n 1 basename
#ls ${FD_CNT}/*.unstranded.bed | wc -l

Tile_K562_hg38_20200905.FADS.stranded_neg.round.bed
Tile_K562_hg38_20200905.FADS.stranded_pos.round.bed
Tile_K562_hg38_20210130.GATA1.stranded_pos.round.bed
Tile_K562_hg38_20210130.MYC.stranded_pos.round.bed


In [21]:
sbatch -p ${NODE} \
    --exclude=dl-01 \
    --cpus-per-task 8 \
    --mem 20G \
    --output ${FD_LOG}/coverage_astarr_peak_macs_input_tmpra.%a.txt \
    --array 0-1 \
    <<'EOF'
#!/bin/bash
### set directories & global variables
source ../config/config_duke.sh
IDX_COLS=(8 9)
IDN_COLS=(Input Output)

### set input and output directory
PREFIX=Tewhey_K562_TileMPRA
FD_CNT=${FD_RES}/${PREFIX}/fragment_count
FD_COV=${FD_RES}/${PREFIX}/coverage_astarrseq_peak_macs_input

FD_REG=${FD_RES}/region/KS91_K562_astarrseq_peak_macs
FN_REG=KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed

IDX_COL=${IDX_COLS[${SLURM_ARRAY_TASK_ID}]}
IDN_COL=${IDN_COLS[${SLURM_ARRAY_TASK_ID}]}

### print start message
timer_start=`date +%s`
echo "Hostname:          " $(hostname)
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}
echo "Time Stamp:        " $(date +"%m-%d-%y+%T")
echo

### init: create output folder if not exist
mkdir -p ${FD_COV}

### loop through files
FP_CNTS=($(ls ${FD_CNT}/*.round.bed))
for FP_INP in ${FP_CNTS[@]}; do
    FN_INP=$(basename ${FP_INP})
    FN_OUT=${FN_INP%.*.*}.${IDN_COL}.bed.gz
    FP_OUT=${FD_COV}/${FN_OUT}
    echo ++++++++++++++++++++++++++++++++++++++
    echo
    
    ### show input file
    echo "Input: " ${FP_INP}
    echo
    echo "show first few lines of input"
    cat ${FP_INP} | head -5
    echo
    
    ### execute
    bedtools map \
        -a ${FD_REG}/${FN_REG} \
        -b ${FP_INP} \
        -o mean \
        -c ${IDX_COL} \
        -f 0.5 \
        -F 0.5 \
        -e \
    | gzip -c > ${FP_OUT}
    
    
    ### show output file
    echo
    echo "Output: " ${FP_OUT}
    echo
    echo "show first few lines of output:"
    zcat ${FP_OUT} | head -5
    echo
done

### print end message
timer=`date +%s`
runtime=$(echo "${timer} - ${timer_start}" | bc -l)
echo
echo 'Done!'
echo "Run Time: $(displaytime ${runtime})"

EOF

Submitted batch job 29160084


**CHECK**

In [20]:
cat ${FD_LOG}/coverage_astarr_peak_macs_input_tmpra.0.txt

Hostname:           x1-01-3.genome.duke.edu
Slurm Array Index:  0
Time Stamp:         10-17-22+10:08:28

++++++++++++++++++++++++++++++++++++++

Input:  /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/Tewhey_K562_TileMPRA/fragment_count/Tile_K562_hg38_20200905.FADS.stranded_neg.round.bed

show first few lines of input
chr11	61787329	61787528	11:61554801-61555000_RC	.	-	-0.882	325.845	176.742
chr11	61787334	61787533	11:61554806-61555005_RC	.	-	-0.832	336.927	189.098
chr11	61787339	61787538	11:61554811-61555010_RC	.	-	-0.391	310.188	236.291
chr11	61787344	61787543	11:61554816-61555015_RC	.	-	-0.768	580.396	341.076
chr11	61787349	61787548	11:61554821-61555020_RC	.	-	-1.093	544.991	255.805

/var/spool/slurmd/job29160078/slurm_script: line 51: /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/Tewhey_K562_TileMPRA/coverage_astarrseq_peak_macs_input/Tile_K562_hg38_20200905.FADS.stranded_neg.Input.bed.gz: No such file or directory

Output:  /gpfs/fs1/data/reddylab/Kuei/ou

In [23]:
cat ${FD_LOG}/coverage_astarr_peak_macs_input_tmpra.1.txt

Hostname:           x1-01-3.genome.duke.edu
Slurm Array Index:  1
Time Stamp:         10-17-22+10:10:28

++++++++++++++++++++++++++++++++++++++

Input:  /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/Tewhey_K562_TileMPRA/fragment_count/Tile_K562_hg38_20200905.FADS.stranded_neg.round.bed

show first few lines of input
chr11	61787329	61787528	11:61554801-61555000_RC	.	-	-0.882	325.845	176.742
chr11	61787334	61787533	11:61554806-61555005_RC	.	-	-0.832	336.927	189.098
chr11	61787339	61787538	11:61554811-61555010_RC	.	-	-0.391	310.188	236.291
chr11	61787344	61787543	11:61554816-61555015_RC	.	-	-0.768	580.396	341.076
chr11	61787349	61787548	11:61554821-61555020_RC	.	-	-1.093	544.991	255.805


Output:  /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/Tewhey_K562_TileMPRA/coverage_astarrseq_peak_macs_input/Tile_K562_hg38_20200905.FADS.stranded_neg.Output.bed.gz

show first few lines of output:
chr1	10015	10442	.
chr1	14253	14645	.
chr1	16015	16477	.
chr1	17237	17772	.
c

In [28]:
PREFIX=Tewhey_K562_TileMPRA
FDIRY=${FD_RES}/${PREFIX}/coverage_astarrseq_peak_macs_input
ls ${FDIRY}

Tile_K562_hg38_20200905.FADS.stranded_neg.Input.bed.gz
Tile_K562_hg38_20200905.FADS.stranded_neg.Output.bed.gz
Tile_K562_hg38_20200905.FADS.stranded_pos.Input.bed.gz
Tile_K562_hg38_20200905.FADS.stranded_pos.Output.bed.gz
Tile_K562_hg38_20210130.GATA1.stranded_pos.Input.bed.gz
Tile_K562_hg38_20210130.GATA1.stranded_pos.Output.bed.gz
Tile_K562_hg38_20210130.MYC.stranded_pos.Input.bed.gz
Tile_K562_hg38_20210130.MYC.stranded_pos.Output.bed.gz


In [33]:
PREFIX=Tewhey_K562_TileMPRA
FDIRY=${FD_RES}/${PREFIX}/coverage_astarrseq_peak_macs_input
FNAME=Tile_K562_hg38_20210130.GATA1.stranded_pos.Input.bed.gz
FPATH=${FDIRY}/${FNAME} 

zcat ${FPATH} | grep chrX | head -n 1400 | tail -n 4

chrX	47809119	47809445	279.4821667
chrX	47814810	47815443	448.15475
chrX	47816459	47818070	447.7993437
chrX	47836113	47837157	357.6077143
grep: write error


In [35]:
PREFIX=Tewhey_K562_TileMPRA
FDIRY=${FD_RES}/${PREFIX}/coverage_astarrseq_peak_macs_input
FNAME=Tile_K562_hg38_20210130.GATA1.stranded_pos.Output.bed.gz
FPATH=${FDIRY}/${FNAME} 

zcat ${FPATH} | grep chrX | head -n 1400 | tail -n 4

chrX	47809119	47809445	5032.433833
chrX	47814810	47815443	2462.516
chrX	47816459	47818070	2854.201719
chrX	47836113	47837157	9407.757762
grep: write error


In [38]:
PREFIX=Tewhey_K562_TileMPRA
FDIRY=${FD_RES}/${PREFIX}/coverage_astarrseq_peak_macs_input
FNAME=Tile_K562_hg38_20210130.MYC.stranded_pos.Output.bed.gz
FPATH=${FDIRY}/${FNAME} 

zcat ${FPATH} | grep chr8 | head -n 8300 | tail -n 4

chr8	127793817	127795585	933.5282157
chr8	127800218	127801557	1336.826154
chr8	127806610	127807136	1023.9795
chr8	127807462	127808467	3273.55085
grep: write error

gzip: stdout: Broken pipe
